In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import data_loader
import numpy as np
import sample_to_chords as s2c


### Definition VAE

In [4]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
print('using',dev)
device = torch.device(dev)
class VAE(nn.Module):
    def __init__(self):
        N_HIDDEN = 400
        N_LATENT = 40
        
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(16*12*7, N_HIDDEN)
        self.fc21 = nn.Linear(N_HIDDEN, N_LATENT)
        self.fc22 = nn.Linear(N_HIDDEN, N_LATENT)
        self.fc3 = nn.Linear(N_LATENT, N_HIDDEN)
        self.fc4 = nn.Linear(N_HIDDEN, 16*12*7)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        soft = nn.Sigmoid()
        return soft(self.fc4(h3).view(-1, 16, 12*7))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 16*12*7))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

using cuda:0


In [5]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar, beta):
    BCE = F.binary_cross_entropy(recon_x.view(-1, 16*12*7), x.view(-1, 16*12*7), reduction='sum')

    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + beta*KLD

In [4]:
def train(epoch):
    model.train()
    train_loss = 0
    beta = epoch/epochs
    for batch_idx, data in enumerate(realbook_dataset):
        data = data.to(device)
        optimizer.zero_grad()
    
    
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar, beta)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), Nchunks,
                100. * batch_idx * len(data)/ Nchunks,
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / Nchunks))

In [5]:
epochs = 10
batch_size = 128
log_interval = 100

In [6]:
realbook_dataset = data_loader.import_dataset()
Nchunks = len(realbook_dataset)
realbook_dataset = torch.split(realbook_dataset, batch_size, 0)
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

Dataset loaded !


### Load model

In [ ]:
model.load_state_dict(torch.load("./model_realbook.pt"))

### Train model

In [7]:
for epoch in range(1, epochs + 1):
    train(epoch)

Train Epoch: 1 [0/394813 (0%)]	Loss: 940.960876
Train Epoch: 1 [12800/394813 (3%)]	Loss: 93.857872
Train Epoch: 1 [25600/394813 (6%)]	Loss: 35.944496
Train Epoch: 1 [38400/394813 (10%)]	Loss: 63.293659
Train Epoch: 1 [51200/394813 (13%)]	Loss: 62.157318
Train Epoch: 1 [64000/394813 (16%)]	Loss: 53.875652
Train Epoch: 1 [76800/394813 (19%)]	Loss: 41.695263
Train Epoch: 1 [89600/394813 (23%)]	Loss: 56.123219
Train Epoch: 1 [102400/394813 (26%)]	Loss: 45.890526
Train Epoch: 1 [115200/394813 (29%)]	Loss: 26.608919
Train Epoch: 1 [128000/394813 (32%)]	Loss: 24.649723
Train Epoch: 1 [140800/394813 (36%)]	Loss: 41.036995
Train Epoch: 1 [153600/394813 (39%)]	Loss: 20.808233
Train Epoch: 1 [166400/394813 (42%)]	Loss: 37.357555
Train Epoch: 1 [179200/394813 (45%)]	Loss: 28.931601
Train Epoch: 1 [192000/394813 (49%)]	Loss: 47.596252
Train Epoch: 1 [204800/394813 (52%)]	Loss: 17.650530
Train Epoch: 1 [217600/394813 (55%)]	Loss: 39.296452
Train Epoch: 1 [230400/394813 (58%)]	Loss: 23.975031
Train E

In [8]:
torch.save(model.state_dict(), "./model_realbook.pt")

### Test

In [10]:
PITCH_LIST = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]
QUALITY_LIST = ["maj", "min", "dim", "maj7", "min7", "7", "dim7"]

def sample_to_chords(sample):
    idx_chords = np.argmax(sample[0,:,:],1)
#     print(idx_chords)
    chords = [PITCH_LIST[int(idx/7)] + ":" + QUALITY_LIST[int(idx%7)] for idx in idx_chords]
    return chords

index_test = 16
test_sample = realbook_dataset[0][index_test]

print("Vérité")
true_sample = test_sample.view(1, 16, -1).numpy()
print(sample_to_chords(true_sample))

print()
print("Par VAE")
recons_test, _, _ = model(test_sample)
print(sample_to_chords(recons_test.detach().numpy()))

Vérité
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:7', 'C:7', 'C:7', 'C:7', 'F:maj', 'F:maj', 'F:maj', 'F:maj', 'F:min7', 'F:min7', 'F:min7', 'F:min7']

Par VAE
['C:maj', 'C:maj', 'C:maj', 'C:maj', 'C:7', 'C:7', 'C:7', 'C:7', 'F:maj', 'F:maj', 'F:maj', 'F:maj', 'F:min7', 'F:min7', 'F:min', 'C:maj']


### Create

In [13]:
with torch.no_grad():
    sample = torch.randn(1, 40).to(device)
    sample = model.decode(sample).cpu()
    sample = sample.numpy()
print(sample.shape)
sample_to_chords(sample)

(1, 16, 84)


['F:maj',
 'F:maj',
 'E:min7',
 'E:min7',
 'A:7',
 'A:7',
 'A:7',
 'D:min7',
 'D:min7',
 'D:min7',
 'D:min7',
 'G:maj',
 'G:maj',
 'G:maj',
 'F:maj',
 'C:maj']

3085